In [1]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
data = pd.read_csv ("../dunnhumby_The-Complete-Journey/dunnhumby - The Complete Journey CSV/transaction_data.csv")
print (data.head ())
data["payed"] = data[["QUANTITY","SALES_VALUE", "RETAIL_DISC", "COUPON_DISC", "COUPON_MATCH_DISC"]].apply(lambda x:x["QUANTITY"] * (x["SALES_VALUE"] + x["RETAIL_DISC"] + x["COUPON_DISC"] + x["COUPON_MATCH_DISC"]),axis=1)


   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  
0       364        -0.60        1631        1          0.0                0.0  
1       364         0.00        1631        1          0.0                0.0  
2       364        -0.30        1631        1          0.0                0.0  
3       364         0.00        1631        1          0.0                0.0  
4       364        -0.39        1631        1          0.0                0.0  


In [3]:
print (data.head ())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  \
0       364        -0.60        1631        1          0.0                0.0   
1       364         0.00        1631        1          0.0                0.0   
2       364        -0.30        1631        1          0.0                0.0   
3       364         0.00        1631        1          0.0                0.0   
4       364        -0.39        1631        1          0.0                0.0   

   payed  
0   0.79  
1   0.82  
2   0.69  
3   1.21  
4   1.11  


In [4]:

rfm_df = data.groupby('household_key').agg({'DAY': 'max',
                                        'BASKET_ID': 'nunique',
                                        'payed': 'sum'})

rfm_df["r_q"] = rfm_df['DAY'].apply (lambda x: 5 if x > 700 else (4 if x > 500 else (3 if x > 300 else (2 if x > 100 else 1))))
rfm_df["f_q"] = rfm_df['BASKET_ID'].apply (lambda x: 5 if x > 200 else (4 if x > 150 else (3 if x > 100 else (2 if x > 50 else 1))))
rfm_df["m_q"] = rfm_df['payed'].apply (lambda x: 5 if x > 4000 else (4 if x > 3000 else (3 if x > 2000 else (2 if x > 1000 else 1))))
rfm_df

,DAY,BASKET_ID,payed,r_q,f_q,m_q
household_key,,,,,,
1,706,86,4166.69,5,2,5
2,668,45,1840.67,4,1,2
3,703,47,137970.32,5,1,5
4,627,30,1496.84,4,1,2
5,703,40,701.06,5,1,1
...,...,...,...,...,...,...
2496,683,63,3083573.55,4,2,5
2497,700,221,1471990.21,4,5,5
2498,710,172,728337.72,5,4,5


In [5]:
rfm_df.rename(columns={'DAY':'recency',
                         'BASKET_ID': 'frequency',
                         'payed': 'monetary value'}, inplace=True)
rfm_df

,recency,frequency,monetary value,r_q,f_q,m_q
household_key,,,,,,
1,706,86,4166.69,5,2,5
2,668,45,1840.67,4,1,2
3,703,47,137970.32,5,1,5
4,627,30,1496.84,4,1,2
5,703,40,701.06,5,1,1
...,...,...,...,...,...,...
2496,683,63,3083573.55,4,2,5
2497,700,221,1471990.21,4,5,5
2498,710,172,728337.72,5,4,5


In [6]:
cltv = data.groupby('household_key').agg({'DAY': 'max',
                                        'QUANTITY': 'sum',
                                        'BASKET_ID': 'nunique',
                                        'payed': 'sum'})

cltv.rename(columns={'DAY':'days',
                         'QUANTITY': 'units',
                         'BASKET_ID': 'transactions',
                         'payed': 'spend'}, inplace=True)

cltv.head()

,days,units,transactions,spend
household_key,,,,
1,706,1997,86,4166.69
2,668,834,45,1840.67
3,703,8540,47,137970.32
4,627,382,30,1496.84
5,703,245,40,701.06


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

rfmc_km_log = np.log(rfm_df)
scaler.fit(rfmc_km_log)
rfmc_km_normalized_log = scaler.transform(rfmc_km_log)
rfmc_km_normalized_log

array([[ 0.23772329,  0.19146978, -0.75277501,  0.62083016, -0.03878696,
         0.6365722 ],
       [-0.1096169 , -0.44784708, -0.96683894, -0.8369307 , -1.21088309,
        -0.89170907],
       [ 0.21098964, -0.40492378,  0.16425432,  0.62083016, -1.21088309,
         0.6365722 ],
       ...,
       [ 0.273192  ,  0.87566165,  0.60017492,  0.62083016,  1.13330918,
         0.6365722 ],
       [ 0.26434359,  0.23634479,  1.13239822,  0.62083016, -0.03878696,
         0.6365722 ],
       [ 0.25548269,  0.46098268,  1.34524954,  0.62083016,  0.64684533,
         0.6365722 ]])

In [13]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3, random_state = 1)
kmeans.fit(rfmc_km_normalized_log)

KMeans(n_clusters=3, random_state=1)

In [14]:
cluster_labels = kmeans.labels_
rfmc_km_k2 = rfm_df.assign(Cluster = cluster_labels)

rfmc_km_k2.groupby('Cluster').agg({'recency':'mean', 
                                  'frequency':'mean',
                                  'monetary value': ['mean','count']})

recency   frequency monetary value      
               mean        mean           mean count
Cluster                                             
0        669.854582   36.972112   2.401355e+05  1004
1        169.352941    4.882353   2.507176e+05    17
2        701.928330  161.785666   5.165616e+06  1479

In [16]:
from cluster1.selfrepresentation import ElasticNetSubspaceClustering

model = ElasticNetSubspaceClustering(n_clusters=3,algorithm='lasso_lars',gamma=50).fit(rfm_df)
cluster_labels = model.labels_

rfmc_km_k2 = rfm_df.assign(Cluster = cluster_labels)

rfmc_km_k2.groupby('Cluster').agg({'recency':'mean', 
                                  'frequency':'mean',
                                  'monetary value': ['mean','count']})

2500


recency   frequency monetary value      
               mean        mean           mean count
Cluster                                             
0        685.397985  103.511335   2.696267e+06   794
1        685.769870  121.134045   3.787749e+06   843
2        685.115875  106.813441   2.956426e+06   863